In [ ]:
# layout.py

import pandas as pd
import dash_bootstrap_components as dbc
from dash import dcc, html

#import dash_table
from dash import dash_table

from app.utils import SYMBOLS

def create_navbar():
    return dbc.Navbar(
        dbc.Container(
            [
                dbc.NavbarBrand("Binance Futures Trading Dashboard", href="/", className="text-white"),
                dbc.Nav(
                    [
                        dbc.NavItem(dbc.NavLink("Global", href="/global", className="text-white")),
                        dbc.NavItem(dbc.NavLink("Place Order", href="/place-order", className="text-white")),
                    ],
                    className="ml-auto",
                    navbar=True,
                ),
            ],
            fluid=True,
        ),
        color="dark",
        dark=True,
        sticky="top",
    )

def create_layout():
    return dbc.Container([
        create_navbar(),
        dbc.Row([
            dbc.Col(html.H1("Binance Futures Trading Dashboard", className="text-center mb-4 text-white"), width=12)
        ], className="mt-4"),
        
        # ------------------------------------------
        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-chart-line me-2"),
                    "Wallet Overviews"
                ], className="text-white"),
                dcc.Interval(id='interval-component', interval=10*1000, n_intervals=0),
                dcc.Graph(id='waterfall-graph', config={'displayModeBar': False}),
            ], width=6),
            dbc.Col(html.Div(id='risk-indicators'), width=6)
        ], className="mt-4"),
        
        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-chart-line me-2"),
                    "Positions Actuelles"
                ], className="text-white"),
                html.Div(id='positions-table', className="text-white")
            ], width=12)
        ], className="mt-4"),
        
        # ------------------------------------------

        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-wallet me-2"),
                    "OHLC"
                ], className="text-white"),
                dcc.Interval(id='interval-component_ohlc', interval=10*1000, n_intervals=0),
                dcc.Graph(id='ohlc-graph', config={'displayModeBar': False})
            ], width=10),  # 70% width
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-hand-holding-usd me-2"),
                    "Placer un Ordre"
                ], className="text-white"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Symbole", className="text-white"),
                        dcc.Dropdown(SYMBOLS, id='order-symbol', value='BTCUSDT', className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Type d'ordre", className="text-white"),
                        dcc.RadioItems(
                            id='order-type',
                            options=[
                                {'label': 'Market', 'value': 'MARKET'},
                                {'label': 'Limit', 'value': 'LIMIT'}
                            ],
                            value='MARKET',
                            className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Quantité/Unit", className="text-white"),
                        dcc.Input(id='order-quantity', type='number', className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Limit Price", className="text-white"),
                        dcc.Input(id='order-price', type='number', className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Cours Actuel", className="text-white"),
                        html.Div(id='current-price', className="text-white mt-2")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Stop Loss", className="text-white"),
                        dcc.Input(id='order-SL', type='number', className="bg-dark text-white")
                    ], width=6),
                    dbc.Col([
                        dbc.Label("Take Profit", className="text-white"),
                        dcc.Input(id='order-TP', type='number', className="bg-dark text-white")
                    ], width=6)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Levier", className="text-white"),
                        dcc.Slider(1, 20, 4, id='order-leverage', className='bg-dark text-white')
                    ], width=12)
                ], className="mb-2"),
                html.Div(id='order-preview', className="text-white mt-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Button("Long", id='order-long', color="success", n_clicks=0, className="me-2")
                    ], width=6),
                    dbc.Col([
                        dbc.Button("Short", id='order-short', color="danger", n_clicks=0, className="me-2")
                    ], width=6)
                ]),
                html.Div(id='order-result', className="text-white mt-2")
            ], width=2)  # 30% width
        ], className="mt-4"),
        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-list-alt me-2"),
                    "Événements"
                ], className="text-white"),
                html.Div(id='events-div', className="text-white")
            ], width=12)
        ], className="mt-4")
    ], fluid=True)


In [ ]:
# callbacks.py

import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from datetime import datetime, timedelta
from app.binance_api import get_positions, get_pnl, place_order, get_price, get_ohlc_data, get_open_orders
from app.db_utils import log_event, get_all_events, update_position_status
from app.utils import dark_template, portfolio_overviews, update_position_state

def register_callbacks(app):
    
    # Wallet overviews
    @app.callback(
        [Output('waterfall-graph', 'figure'), Output('risk-indicators', 'children')],
        [Input('interval-component', 'n_intervals'), Input('order-symbol', 'value')]
    )
    def update_wallet(n, symbol):
        wallet_data = get_pnl()
        r_risk , waterfall_fig = portfolio_overviews(wallet_data)
        
        # Indicateurs de suivi et de risque
        risk_indicators = html.Div([
            html.H5("Indicateurs de Suivi et de Risque"),
            html.P(f"Leverage Ratio = {r_risk['leverage_ratio']}%"),
            html.P(f"Maintenance Margin Ratio = {r_risk['maint_margin_ratio']}%"),
            html.P(f"Risk-Reward Ratio = {r_risk['risk_reward_ratio']}%"),
        ])
        return waterfall_fig, risk_indicators
    
    # Update positions
    @app.callback(
        [Output('positions-table', 'children'), Output('ohlc-graph', 'figure')],
        [Input('interval-component', 'n_intervals'), Input('order-symbol', 'value')]
    )
    def update_position(n, symbol):
        positions_data = get_positions()
        open_orders = get_open_orders()
        
        start_date = datetime.now() - timedelta(days=7)
        end_date = datetime.now()
        interval = '1h'
        data = get_ohlc_data(symbol, start_date, end_date, interval)
        
        positions, fig_ohlc = update_position_state(symbol=symbol, positions_data=positions_data, open_orders_data=open_orders,
                         get_price=get_price, ohlc_data=data)
        
        # Tableau des Positions Actuelles
        df_positions = pd.DataFrame(positions)
        positions_table = dbc.Table.from_dataframe(df_positions, striped=True, bordered=True, hover=True)
        return positions_table, fig_ohlc
    
    
    # Callback pour fermer une position
    @app.callback(
        Output('positions-table', 'data'),
        [Input({'type': 'close-position-button', 'index': dash.dependencies.ALL}, 'n_clicks')],
        [State('positions-table', 'data')]
    )
    def close_position(n_clicks, rows):
        ctx = dash.callback_context
        if not ctx.triggered:
            return rows

        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        row_index = int(button_id.split('.')[-1])

        # Fermer la position correspondante
        if rows[row_index]['status'] == 'open':
            update_position_status(rows[row_index]['id'], 'closed')
            rows[row_index]['status'] = 'closed'

        return rows
    
    # Callback pour prévisualiser l'ordre
    @app.callback(
        Output('order-preview', 'children'),
        [Input('order-symbol', 'value'), Input('order-quantity', 'value'), Input('order-leverage', 'value')]
    )
    def preview_order(symbol, quantity, leverage):
        if symbol and quantity and leverage:
            price_data = get_price(symbol)
            if 'price' in price_data:
                price = float(price_data['price'])
                total = price * quantity * leverage
                return f"Prix actuel : {price} USD, Total : {total} USD (avec un levier de {leverage}x)"
        return ""

    # Callback pour afficher ou masquer le champ "Limit Price"
    @app.callback(
        Output('order-price', 'style'),
        [Input('order-type', 'value')]
    )
    def show_hide_limit_price(order_type):
        if order_type == 'LIMIT':
            return {'display': 'block'}
        return {'display': 'none'}

    # Callback pour mettre à jour le cours du symbole sélectionné
    @app.callback(
        Output('current-price', 'children'),
        [Input('order-symbol', 'value')]
    )
    def update_current_price(symbol):
        price_data = get_price(symbol)
        if 'price' in price_data:
            price = float(price_data['price'])
            return f"{price} USD"
        return "Erreur lors de la récupération du prix"

    
# ------------------------------------- Place Order -----------------------
    # Fonction de validation de l'entrée pour les ordres
    def validate_order_input(symbol, order_type, quantity, leverage, price):
        if not symbol or not order_type or not quantity or not leverage or (order_type == 'LIMIT' and not price):
            return "Tous les champs sont obligatoires pour les ordres limit", "alert alert-danger"
        if quantity <= 0 or leverage <= 0 or (price and price <= 0):
            return "La quantité, le levier et le prix doivent être des valeurs positives", "alert alert-danger"
        #if symbol and order_type and quantity and leverage and (order_type == 'MARKET' and price is None):
        #    return f"symbol : {symbol} - quantity : {quantity} - price : {price}", "OK continuons"
        return None, None

    # Fonction pour journaliser l'événement et retourner le résultat de l'ordre
    def log_and_return_order_result(result, event_time, symbol, side, order_type, quantity, price):
        if 'orderId' in result:
            log_event('ORDER_PLACED', event_time, symbol, side, order_type, quantity, price, 'SUCCESS')
            return f"Ordre placé avec succès : {result}", "alert alert-success"
        else:
            log_event('ORDER_FAILED', event_time, symbol, side, order_type, quantity, price, 'FAILURE')
            return f"Erreur lors du placement de l'ordre : {result}", "alert alert-danger"

    # Callback pour placer un ordre Long / Short
    @app.callback(
        [Output('order-result', 'children'), Output('order-result', 'className')],
        [Input('order-long', 'n_clicks'), Input('order-short', 'n_clicks')],
        [State('order-symbol', 'value'),
         State('order-type', 'value'),
         State('order-quantity', 'value'),
         State('order-leverage', 'value'),
         State('order-price', 'value')]
    )
    def place_order_callback(n_clicks_long, n_clicks_short, symbol, order_type, quantity, leverage, price):
        ctx = dash.callback_context

        if not ctx.triggered:
            return "", ""

        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'order-long':
            side = "BUY"
        elif button_id == 'order-short':
            side = "SELL"
        else:
            return "", ""

        error_message, error_class = validate_order_input(symbol, order_type, quantity, leverage, price)
        if error_message:
            return error_message, error_class

        result = place_order(symbol, side, order_type, quantity, leverage, price)
        event_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        return log_and_return_order_result(result, event_time, symbol, side, order_type, quantity, price)

    # Callback pour afficher les événements
    @app.callback(
        Output('events-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def display_events(n):
        events = get_all_events()
        df_events = pd.DataFrame(events, columns=['ID', 'Type', 'Time', 'Symbol', 'Side', 'Order Type', 'Quantity', 'Price', 'Result'])
        return dbc.Table.from_dataframe(df_events, striped=True, bordered=True, hover=True)


In [ ]:
# config.py

import os
from dotenv import load_dotenv
#load_dotenv()

#API_KEY = os.getenv('BINANCE_API_KEY')
#API_SECRET = os.getenv('BINANCE_API_SECRET')


# Ced_Syst
#API_KEY = 'C8Lw6mJh4CNYQXVIgdRAv64S5bQzh1RyNBQJNL3C2roe8rsxyTtN8EfB4faadhD3'
#API_SECRET = 'aokTvHmhUOFnhOlSNp2a7VPh1NVfkydrFHgdqRArKjuoz2AXYJTBOsNsyxhPsXs8'

# Test
API_KEY = "d727ada381ac80bb187e04ca361e0e0f6ba5f6fc22d3cbf610c17c32d936657e"
API_SECRET = "2f33bf9f838e705446915b60dfa4440c2303d478a7a726536bd0b264c2b1ec45"
URL = "https://testnet.binancefuture.com"

#BASE_URL = 'https://fapi.binance.com'
BASE_URL = "https://testnet.binancefuture.com"





In [ ]:
from dash.dependencies import Input, Output, State
from dash import dcc, html
import dash_bootstrap_components as dbc
import dash
import plotly.graph_objs as go
from datetime import datetime
import pandas as pd
from app.api.account import Account
from app.api.order import Order



account = Account()

def register_callbacks(app):

    @app.callback(
        Output('ohlc-graph', 'figure'),
        Input('interval-component', 'n_intervals'),
        State('symbol-dropdown', 'value')
    )
    def update_ohlc_graph(n_intervals, symbol):
        ohlc_data = account.get_ohlc_data(symbol)
        df = pd.DataFrame(ohlc_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        fig = go.Figure(data=[go.Ohlc(
            x=df['timestamp'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close']
        )])
        
        return fig

    @app.callback(
        Output('order-response', 'children'),
        [Input('long-button', 'n_clicks'), Input('short-button', 'n_clicks')],
        [State('symbol-dropdown', 'value'),
         State('quantity-input', 'value'),
         State('price-input', 'value'),
         State('stop-loss-input', 'value'),
         State('take-profit-input', 'value'),
         State('leverage-dropdown', 'value'),
         State('order-type-radio', 'value')]
    )
    def place_order(long_clicks, short_clicks, symbol, quantity, price, stop_loss, take_profit, leverage, order_type):
        ctx = dash.callback_context
        if not ctx.triggered:
            return ''
        
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        side = 'BUY' if button_id == 'long-button' else 'SELL'
        
        order = Order(symbol)
        order.update_leverage(leverage)
        
        if side == 'BUY':
            response = order.open_long(quantity, leverage, stop_loss, take_profit)
        else:
            response = order.open_short(quantity, leverage, stop_loss, take_profit)
        
        if response.get('code') != 2:
            return dbc.Alert(f"Order placed: {response}", color="success")
        else:
            return dbc.Alert(f"Error: {response['msg']}", color="danger")

    @app.callback(
        Output('positions-output', 'children'),
        Input('view-positions-button', 'n_clicks')
    )
    def view_positions(n_clicks):
        if n_clicks > 0:
            positions = account.get_positions()
            df = pd.DataFrame(positions)
            return dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True, responsive=True)
        return ""

    @app.callback(
        Output('orders-output', 'children'),
        Input('view-orders-button', 'n_clicks')
    )
    def view_orders(n_clicks):
        if n_clicks > 0:
            orders = account.get_open_orders()
            df = pd.DataFrame(orders)
            return dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True, responsive=True)
        return ""

    @app.callback(
        [Output('wallet-info', 'children'), Output('waterfall-graph', 'figure')],
        Input('interval-component', 'n_intervals')
    )
    def update_wallet_info(n_intervals):
        pnl = account.get_pnl()
        totalWalletBalance = float(pnl['totalWalletBalance'])
        totalInitialMargin = float(pnl['totalInitialMargin'])
        totalMaintMargin = float(pnl['totalMaintMargin'])
        totalUnrealizedProfit = float(pnl['totalUnrealizedProfit'])
        totalMarginBalance = float(pnl['totalMarginBalance'])
        availableBalance = float(pnl['availableBalance'])
        maxWithdrawAmount = float(pnl['maxWithdrawAmount'])
        
        r_risk, waterfall_fig = portfolio_overviews(pnl)
        
        return (
            html.Div([
                html.P(f"Total Wallet Balance: {totalWalletBalance}"),
                html.P(f"Total Initial Margin: {totalInitialMargin}"),
                html.P(f"Total Maintenance Margin: {totalMaintMargin}"),
                html.P(f"Total Unrealized Profit: {totalUnrealizedProfit}"),
                html.P(f"Total Margin Balance: {totalMarginBalance}"),
                html.P(f"Available Balance: {availableBalance}"),
                html.P(f"Max Withdraw Amount: {maxWithdrawAmount}")
            ]),
            waterfall_fig
        )

def portfolio_overviews(portfolio_data):
    totalWalletBalance = float(portfolio_data['totalWalletBalance'])
    totalInitialMargin = float(portfolio_data['totalInitialMargin'])
    totalMaintMargin = float(portfolio_data['totalMaintMargin'])
    totalUnrealizedProfit = float(portfolio_data['totalUnrealizedProfit'])
    totalMarginBalance = float(portfolio_data['totalMarginBalance'])
    availableBalance = float(portfolio_data['availableBalance'])
    
    waterfall_fig = waterfall(totalWalletBalance, totalInitialMargin, totalUnrealizedProfit, availableBalance)
    return {}, waterfall_fig

def waterfall(totalWalletBalance, totalInitialMargin, totalUnrealizedProfit, availableBalance):
    waterfall_fig = go.Figure(
        go.Waterfall(
            name="20", 
            orientation="v",
            measure=["relative", "relative", "relative", "total"],
            x=["Wallet Balance", "Initial Margin", "UPnL", "Available Balance"],
            text=[
                f"${totalWalletBalance:.3f}", 
                f"${totalInitialMargin:.3f}", 
                f"${totalUnrealizedProfit:.3f}", 
                f"${availableBalance:.3f}"
            ],
            y=[
                totalWalletBalance, 
                -totalInitialMargin, 
                totalUnrealizedProfit, 
                availableBalance
            ],
            connector={"line": {"color": "rgb(63, 63, 63)"}},
            decreasing={"marker": {"color": "Maroon", "line": {"color": "red", "width": 2}}},
            increasing={"marker": {"color": "Teal"}}
        )
    )
    return waterfall_fig


In [ ]:
#binance_api.py

import requests
import time
import hashlib
import hmac
import os
import pandas as pd
from decimal import Decimal

from app.config import API_KEY, API_SECRET, BASE_URL


# Test
API_KEY = "d727ada381ac80bb187e04ca361e0e0f6ba5f6fc22d3cbf610c17c32d936657e"
API_SECRET = "2f33bf9f838e705446915b60dfa4440c2303d478a7a726536bd0b264c2b1ec45"
URL = "https://testnet.binancefuture.com"

#BASE_URL = 'https://fapi.binance.com'
BASE_URL = "https://testnet.binancefuture.com"



# --------------------------------------------   Base  -----------------------------------------------

def create_signature(query_string, secret):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def binance_request(endpoint, params):
    params['timestamp'] = int(time.time() * 1000)
    query_string = '&'.join([f"{key}={value}" for key, value in sorted(params.items())])
    signature = create_signature(query_string, API_SECRET)
    headers = {
        'X-MBX-APIKEY': API_KEY
    }
    response = requests.get(f"{BASE_URL}{endpoint}?{query_string}&signature={signature}", headers=headers)
    return response.json()

def binance_post_request(endpoint, params):
    params['timestamp'] = int(time.time() * 1000)
    query_string = '&'.join([f"{key}={value}" for key, value in sorted(params.items())])
    signature = create_signature(query_string, API_SECRET)
    headers = {
        'X-MBX-APIKEY': API_KEY
    }
    response = requests.post(f"{BASE_URL}{endpoint}?{query_string}&signature={signature}", headers=headers)
    return response.json()

# ------------------------------------------------ End Base  -------------------------------------------

# ------------------------------------------------ OHLC Data  -------------------------------------------
def get_ohlc_data(symbol, start_date, end_date, interval):
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": int(start_date.timestamp() * 1000),
        "endTime": int(end_date.timestamp() * 1000),
        "limit": 500
    }
    #return binance_request('/fapi/v1/indexPriceKline', params)
    return binance_request('/fapi/v1/klines', params)

# ------------------------------------------------ End OHLC Data  -------------------------------------------

# ------------------------------------------------ Account  -------------------------------------------
def get_price(symbol):
    params = {'symbol': symbol}
    return binance_request('/fapi/v1/ticker/price', params)

def get_positions():
    return binance_request('/fapi/v2/positionRisk', {})

def get_pnl():
    return binance_request('/fapi/v2/account', {})


# ------------------------------------------------ End Account  -------------------------------------------


# ---------------------------------------------------- Open Order ---------------------------------------
def get_mark_price(symbol):
    params = {"symbol": symbol}
    return binance_request('/fapi/v1/premiumIndex', params)

def get_open_orders():
    return binance_request('/fapi/v1/openOrders', {})

def get_open_orders_count(symbol):
    orders = get_open_orders()
    return sum(1 for order in orders if order['symbol'] == symbol)

def get_current_position(symbol):
    positions = get_positions()
    position = next((p for p in positions if p['symbol'] == symbol), None)
    return float(position['positionAmt']) if position else 0.0


def get_filters(symbol):
    response = binance_request('/fapi/v1/exchangeInfo', {})
    symbol_info = next((item for item in response['symbols'] if item['symbol'] == symbol), None)
    if symbol_info:
        filters = {f['filterType']: f for f in symbol_info['filters']}
        return filters
    return None

# Market DEPTH
def get_order_book(symbol, limit=100):
    params = {
        'symbol': symbol,
        'limit': limit
    }
    return binance_request('/fapi/v1/depth', params)

# VOLUME
def get_ticker(symbol):
    params = {
        'symbol': symbol
    }
    return binance_request('/fapi/v1/ticker/24hr', params)

# Analyse de la liquidite
def analyze_liquidity(symbol):
    order_book = get_order_book(symbol)
    ticker = get_ticker(symbol)
    
    #Spread
    if len(order_book['bids']) > 0 and len(order_book['asks']) > 0:
        best_bid = float(order_book['bids'][0][0])
        best_ask = float(order_book['asks'][0][0])
        spread = (best_ask - best_bid) / best_bid
    else:
        spread = 10
    
    # Volume
    volume = float(ticker['volume'])
    
    #Depth
    bids = pd.DataFrame(order_book['bids'], columns=['price', 'quantity'], dtype=float)
    asks = pd.DataFrame(order_book['asks'], columns=['price', 'quantity'], dtype=float)
    
    total_bids = bids['quantity'].sum()
    total_asks = asks['quantity'].sum()
    
    # Règles de décision
    rules = {
        'spread': spread < 0.01,  # Spread < 1%
        'volume': volume > 1000,  # Volume > 1000 unités
        'depth': total_bids > 500 and total_asks > 500  # Profondeur significative
    }
    is_liquid = all(rules.values())
    print(symbol)
    return {
        'symbol': symbol,
        'spread': spread,
        'volume': volume,
        'total_bids': total_bids,
        'total_asks': total_asks,
        'is_liquid': is_liquid,
        'rules': rules
    }

def validate_order(symbol, quantity, side, price=None):
    filters = get_filters(symbol)
    if not filters:
        return {'code': -1, 'msg': 'No filters found for the symbol'}
    
    msg = {}
    sugg = {}
    
    # LOT_SIZE
    lot_size = filters.get('LOT_SIZE')
    if lot_size:
        min_qty = lot_size['minQty']
        max_qty = lot_size['maxQty']
        step_size = lot_size['stepSize']
        
        if quantity < float(min_qty):
            msg["minQty"] = f"Invalid Qty. Must be > {min_qty}"
            sugg["qty"] = float(min_qty)
        if quantity > float(max_qty):
            msg["maxQty"] = f"Invalid Qty. Must be < {max_qty}"
            sugg["qty"] = float(max_qty)
        if (Decimal(str(quantity)) - Decimal(min_qty)) % Decimal(step_size) != 0:
            msg["stepSize"] = f"Invalid stepSize"
            r = (Decimal(str(quantity)) - Decimal(min_qty)) % Decimal(step_size)
            sugg["qty"] = float(Decimal(str(quantity)) - r)
    
    # PRICE_FILTER
    price_filter = filters.get("PRICE_FILTER")
    if price_filter and price:
        min_price = price_filter["minPrice"]
        max_price = price_filter["maxPrice"]
        tick_size = price_filter["tickize"]
        if price < float(min_price):
            msg["minPrice"] = f"Invalid Price. Must be > {min_price}"
        if price > float(max_price):
            msg["maxPrice"] = f"Invalid Price. Must be < {max_price}"
        if (Decimal(str(price)) - Decimal(min_price)) % Decimal(tick_size) != 0:
            msg["tickSize"] = f"Invalid Tick Size"
    
     # PERCENT_PRICE
    percent_price = filters.get('PERCENT_PRICE')
    if percent_price and price:
        multiplierUp = percent_price.get('multiplierUp')
        multiplierDown = percent_price.get('multiplierDown')
        multiplierUPDec = percent_price.get('multiplierDecimal')
        mark_price = get_mark_price(symbol)['markPrice']
        
        if (side == "BUY") and (price > float(mark_price )*float(multiplierUp)):
            msg["percentPrice"] = f"BUY : Invalid"
        if (side == "SELL") and (price < float(mark_price) * float(multiplierDown)):
            msg["percentPrice"] = f"SELL : Invalid"
            
    
    # MARKET_LOT_SIZE
    market_lot_size = filters.get('MARKET_LOT_SIZE')
    if market_lot_size:
        m_min_qty = market_lot_size['minQty']
        m_max_qty = market_lot_size['maxQty']
        m_step_size = market_lot_size['stepSize']
        if quantity < float(m_min_qty):
            msg["MminQty"] = f"Invalid Qty. Must be > {m_min_qty}"
            sugg["Mqty"] = float(m_min_qty)
        if quantity > float(m_max_qty):
            msg["MmaxQty"] = f"Invalid Qty. Must be < {m_max_qty}"
            sugg["Mqty"] = float(m_max_qty)
        if (Decimal(str(quantity)) - Decimal(m_min_qty)) % Decimal(m_step_size) != 0:
            msg["MstepSize"] = f"Invalid stepSize"
            r = (Decimal(str(quantity)) - Decimal(m_min_qty)) % Decimal(m_step_size)
            sugg["Mqty"] = float(Decimal(str(quantity)) - r)
            
    if msg != {}:
        print(msg)
        print(sugg)
    
    return msg, sugg
    
    
def place_order(symbol, side, order_type, quantity, price=None, stop_price=None):
    validation1 = validate_order(symbol, quantity, price)
    
    liquidity = analyze_liquidity(symbol)
    validation2 = liquidity["is_liquid"]
    
    if (validation1 == ({}, {})) and (validation2 is True):
        params = {
            'symbol': symbol,
            'side': side,
            'type': order_type,
            'quantity': quantity
        }
        if order_type == "MARKET":
            return binance_post_request('/fapi/v1/order', params)
        
        if order_type == "LIMIT":
            params['price'] = price
            params['stopPrice'] = stop_price
            
    else:
        return {"code" : 2, "msg" : liquidity}
        
# ---------------------------------------------------------  End Order ----------------------------------


